Three Steps for Deep Learning:
- Step 1: define a set of function(Neural Neywork)
- Step 2: goodness of function
- Step 3: pick the best function

## Neural Network

**Neuron**
$$
    z=a_1w_1+...+a_kw_k+...+a_Kw_K+b
$$

![Neuron](images/image1-1.png)

其中，$\sigma(z)=\frac{1}{1+e^{-z}}$

不同的连接会导致不同的网络结构。神经元的权重和bias有不同的值。

Weights and bias are network parameters $\theta$.

Given parameters $\theta$, define a function.Given network structure, define $\underline{\text{ a function set}}$.

![Fully-Connect](images/image1-2.png)

**Why we need deep?**
- **分层特征抽象**:每一层学习不同层级的特征，低层捕捉简单模式（如边缘、纹理），高层则组合成更复杂的概念（如面部、物体）。
- **表达能力指数级提升**:同等参数规模下，深层网络可以用更少的神经元表示更复杂的函数；而浅层网络要达到相同效果往往需要指数级的宽度。
- **参数效率与重用**:深度结构通过共享和复用中间表示，大大减少了冗余参数，提高了训练和推理的效率。
- **感受野扩大**:更多层数能让网络在高层看到更大范围的输入信息，有助于捕捉长程依赖和全局特征。


In [ ]:
import torch
import torch.nn as nn

# 假设我们有 K=3 个输入
K = 3
input_features = torch.tensor([0.5, 0.1, -2.0]) # 对应图中的 a_1, ..., a_K

# 在 PyTorch 中，一个神经元可以被看作是一个输入维度为 K、输出维度为 1 的线性层
# nn.Linear(in_features, out_features) 会自动为我们创建权重(weights)和偏置(bias)
# in_features: 输入特征的数量 (K)
# out_features: 输出特征的数量 (对于单个神经元，是1)
neuron = nn.Linear(in_features=K, out_features=1)

# 我们可以查看这个神经元随机初始化的权重和偏置
# neuron.weight 对应图中的 w_1, ..., w_K
# neuron.bias 对应图中的 b
print(f"神经元的权重 (w): {neuron.weight.data}")
print(f"神经元的偏置 (b): {neuron.bias.data}\n")

# 1. 计算加权和 z = a_1*w_1 + ... + a_K*w_K + b
# PyTorch 的线性层会自动完成这个计算
z = neuron(input_features)
print(f"加权和 (z): {z.data}")

# 2. 应用激活函数 a = σ(z)
# 图中提到的 Sigmoid 函数 σ(z) = 1 / (1 + e^-z)
activation_function = nn.Sigmoid()
output_a = activation_function(z)

print(f"经过 Sigmoid 激活后的最终输出 (a): {output_a.data}")

神经元的权重 (w): tensor([[ 0.3316,  0.2185, -0.4302]])
神经元的偏置 (b): tensor([0.5293])

加权和 (z): tensor([1.5773])
经过 Sigmoid 激活后的最终输出 (a): tensor([0.8288])


In [ ]:
import torch
import torch.nn as nn

# 定义网络结构的一些超参数
input_size = 10   # 输入层特征数量，对应图中的 x_1, ..., x_N (假设 N=10)
hidden_size1 = 32 # 第一个隐藏层的神经元数量
hidden_size2 = 16 # 第二个隐藏层的神经元数量
output_size = 5   # 输出层特征数量，对应图中的 y_1, ..., y_M (假设 M=5)
batch_size = 4    # 假设我们一次处理4个样本

# 使用 nn.Sequential 来构建一个深度网络，这是一种方便的方式来堆叠层
# 每一层都是由多个神经元组成的
model = nn.Sequential(
    # 输入层 -> 第一个隐藏层
    nn.Linear(input_size, hidden_size1),
    nn.Sigmoid(), # 第一个隐藏层的激活函数
    
    # 第一个隐藏层 -> 第二个隐藏层 (这就是“深”的体现)
    nn.Linear(hidden_size1, hidden_size2),
    nn.Sigmoid(), # 第二个隐藏层的激活函数
    
    # 第二个隐藏层 -> 输出层
    nn.Linear(hidden_size2, output_size)
)

print("构建的深度神经网络结构:")
print(model)

# 创建一个符合网络输入要求的随机数据（模拟输入）
# 形状为 (batch_size, input_size)
dummy_input = torch.randn(batch_size, input_size)
print(f"\n模拟输入数据的形状: {dummy_input.shape}")

# 将数据传入网络，得到输出
output = model(dummy_input)
print(f"网络输出的形状: {output.shape}")
print(f"网络输出示例: \n{output.data}")

构建的深度神经网络结构:
Sequential(
  (0): Linear(in_features=10, out_features=32, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=32, out_features=16, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=16, out_features=5, bias=True)
)

模拟输入数据的形状: torch.Size([4, 10])
网络输出的形状: torch.Size([4, 5])
网络输出示例: 
tensor([[ 0.1924, -0.1882,  0.3880, -0.0847,  0.5270],
        [ 0.1899, -0.2070,  0.4059, -0.0945,  0.5419],
        [ 0.2137, -0.2015,  0.4048, -0.1016,  0.5235],
        [ 0.2185, -0.1948,  0.3919, -0.0683,  0.5412]])


**一个经典案例：手写字识别**

Training Data: Preparing training data: images and their labels.<span style="color: red;">The learning target is defined on the training data.

Output layer: In general, the output of network can be any value.May not be easy intepret.

Softmax Layer:$$y_k=\frac{e^{z_k}}{\sum\limits_{j=1}^{n}e^{z_j}}$$

Loss: Loss can be square error or cross entropy between the network output and target.

A good function should make the loss of all examples as small as possible.$\Longrightarrow$ Find a function in function set that minimizes total loss $L$ $\Longrightarrow$ Find the network parameters $\theta^*$ that minimizes total loss $L$.

**How to pick the best function?**

*Gradient Descent*: 

Network parameters $\theta=\{w_1,w_2,...,b_1,b_2,...\}$. Pick an initial value for $w$.
Compute $\frac{\partial L}{\partial w}$,
$$w\leftarrow w-\eta \frac{\partial L}{\partial w}$$
Repeat,util $\frac{\partial L}{\partial w}$ is approximately small (when update is little),
$\eta$ is call "*learning rate*"

Gradient Descent is the "learning" of machines in deep learning, even alpha go using this approach.

**Backpropagation**: an efficient way to compute $\frac{\partial L}{\partial w}$ in neural network.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# --- 准备工作 ---
# 1. 定义一个用于分类的网络 (例如，手写数字识别有10个类别)
input_dim = 784  # 假设是 28x28 的手写数字图片，拉平后是 784
hidden_dim = 128 # 隐藏层维度
output_dim = 10  # 输出维度 (0-9 共10个类别)

# 我们使用和上一步类似的网络结构
model = nn.Sequential(
    nn.Linear(input_dim, hidden_dim),
    nn.Sigmoid(),
    nn.Linear(hidden_dim, output_dim)
)

# 2. 准备模拟的训练数据 (Training Data) 和标签 (Labels)
batch_size = 4
# 模拟4张拉平的图片
mock_images = torch.randn(batch_size, input_dim) 
# 模拟这4张图片对应的真实标签 (The learning target)
mock_labels = torch.tensor([1, 7, 3, 5]) # 比如第一张图是数字1, 第二张是7...

# 3. 定义损失函数 (Loss)
# 图中提到了 cross entropy loss。在PyTorch中，nn.CrossEntropyLoss 内部已经包含了 Softmax 操作。
# 它会先对模型的输出计算 Softmax，再计算交叉熵损失。
loss_function = nn.CrossEntropyLoss()

# 4. 定义优化器 (Optimizer)，实现梯度下降 (Gradient Descent)
# 我们将模型的参数 (θ = {w_1, b_1, ...}) 和学习率 (learning rate, η) 传入优化器
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# --- 训练的核心循环 (演示一步) ---

# 在训练前，需要将已有梯度清零
optimizer.zero_grad()

# a. 前向传播：将数据输入网络，得到输出
# output 在这里可以看作是 z, 即进入 Softmax 层之前的值
outputs = model(mock_images)

# b. 计算损失：比较网络输出和真实标签，计算 Loss
# loss_function 会自动对 outputs 应用 Softmax
loss = loss_function(outputs, mock_labels)
print(f"计算出的当前批次的损失 (Loss): {loss.item()}")

# c. 反向传播 (Backpropagation)：计算损失函数关于每个参数(w, b)的梯度 (∂L/∂w)
# 这是 PyTorch 自动完成的魔法一步
loss.backward()

# d. 更新参数：优化器根据梯度更新网络参数 (w ← w - η * ∂L/∂w)
# 这就是梯度下降的步骤
optimizer.step()

# --- 验证一下 ---
# 我们可以再次计算损失，看看参数更新后，对于同一批数据，损失是否降低
print("\n--- 参数更新后 ---")
new_outputs = model(mock_images)
new_loss = loss_function(new_outputs, mock_labels)
print(f"更新参数后，在同一数据上的新损失: {new_loss.item()}")
print("可以看到损失降低了，说明网络正在向正确的方向学习。")

计算出的当前批次的损失 (Loss): 2.2396841049194336

--- 参数更新后 ---
更新参数后，在同一数据上的新损失: 2.16135835647583
可以看到损失降低了，说明网络正在向正确的方向学习。
